In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
#Histogram Equalization
src = np.array([[0,0,0,0],
                [1,1,3,5],
                [6,1,1,3],
                [4,3,1,7],
               ], dtype=np.uint8)
#3비트를 8bit로 늘렸기 때문에 값이 8비트로 나온다
dst = cv2.equalizeHist(src)
print('dst =', dst)

In [ ]:
src = cv2.imread('../data/lena.jpg', cv2.IMREAD_GRAYSCALE)
dst = cv2.equalizeHist(src)
cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#
src = cv2.imread('../data/lena.jpg')

#1 h s v 색상 명도 채도, v가 밝기
#hsv로 하니까 좀 까매짐. 레나가 때가 낌
hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)
h, s, v = cv2.split(hsv)
#밝기값올리기
v2 = cv2.equalizeHist(v)
hsv2 = cv2.merge([h, s, v2])
dst = cv2.cvtColor(hsv2, cv2.COLOR_HSV2BGR)

#2
#yCrCb_ y는 휘도라고 해서 밝기값.
yCrCb = cv2.cvtColor(src, cv2.COLOR_BGR2YCrCb)
y, Cr, Cb = cv2.split(yCrCb)
y2 = cv2.equalizeHist(y)
yCrCb2 = cv2.merge([y2, Cr, Cb])
dst2 = cv2.cvtColor(yCrCb2, cv2.COLOR_YCrCb2BGR)

cv2.imshow('src',src)
cv2.imshow('dst',dst)
cv2.imshow('dst2',dst2)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
#적응형 히스토그램 평활화
src = cv2.imread('../data/tsukuba_l.png', cv2.IMREAD_GRAYSCALE)
cv2.imshow('src', src)

dst1=cv2.equalizeHist(src)
cv2.imshow('dst1', dst1)
#그리드 사이즈를 1 by 1로 했기 때문에 일반적으로 histogram equalization src와 같음
clahe = cv2.createCLAHE(clipLimit=40, tileGridSize=(1,1))
dst2 = clahe.apply(src)
cv2.imshow('dst2', dst2)
#밝은 부분은 너무 밝게 나오고 그런 경우에는 결국 CLAHE를 한다.
clahe2 = cv2.createCLAHE(clipLimit=40, tileGridSize=(8,8))
dst3 = clahe.apply(src)
cv2.imshow('dst3', dst3)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#histogram backprojection
src = np.array([[0,0,0,0],
                [1,1,3,5],
                [6,1,1,3],
                [4,3,1,7],
               ], dtype=np.uint8)

hist = cv2.calcHist(images=[src], channels=[0], mask=None, histSize=[4], ranges=[0,8])
print('hist =', hist)

backP = cv2.calcBackProject([src], [0], hist,[0,8], scale=1)
print('backP =', backP)

In [ ]:
#1
#프룻 이미지 읽고 hsv로 바꾼다. (v채널(밝기값)을 가지고 백프로젝션을 할것)
src=cv2.imread('../data/fruits.jpg')
hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)
h, s, v = cv2.split(hsv)

#2
#roi설정하기 (이미지에서 roi의 색상과 유사한 영역을 찾고자 함)
roi=cv2.selectROI(src)
print('roi =', roi)
roi_h= h[roi[1]:roi[1]+roi[3], roi[0]:roi[0]+roi[2]]
hist = cv2.calcHist(images=[roi_h], channels=[0], mask=None, histSize=[64], ranges=[0,256])
backP = cv2.calcBackProject([h.astype(np.float32)], [0], hist,[0,256], scale=1.0)

#3
#내림차순 정렬 - 빈도수가 가장 높은 애가 맨 처음에 있을거고 그다음으로 낮아짐
hist=cv2.sort(hist, cv2.SORT_EVERY_COLUMN+cv2.SORT_DESCENDING)
print(f"hist.shape={hist.shape}")
#print(f"hist={hist}")
k=1
#임계값을 맨 처음값으로 쓰겠다 (제일 큰 값)
T=hist[k][0] - 1 # threshold
print('T=',T)
#맨 처음에 roi로 고른 색과 가장 유사한 색인 영역만 남음
ret, dst = cv2.threshold(backP, T, 255, cv2.THRESH_BINARY)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np

src=cv2.imread('../data/bckproj.bmp', cv2.IMREAD_GRAYSCALE)
hist = cv2.calcHist(images=[src], channels=[0], mask=None, histSize=[256], ranges=[0,256])

backP = cv2.calcBackProject([src.astype(np.float32)], [0], hist,[0,256], scale=1)

min_val,max_val,min_loc,max_loc = cv2.minMaxLoc(backP)
print('Gmin = ', min_val)
print('Gmac = ', max_val)

img_norm = cv2.normalize(backP, None, 0, 255, cv2.NORM_MINMAX)
dst = cv2.convertScaleAbs(img_norm)
cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()